In [1]:
import numpy as np

In [2]:
from collections import OrderedDict


In [3]:
foo = OrderedDict()
bar = OrderedDict()

def setdict(key, val):
    foo[key] = val
    bar[key] = str(val)

setdict('b', 6)
setdict('a', 5)
setdict('c', 12)

print(foo, '\n', bar)

setdict('a', 10)

print(foo, '\n', bar)

foo.pop('a')

for k in foo:
    print(k, foo[k])

x = 5 if True else 6
print(x)

x = 5 if False else 6
print(x)

OrderedDict([('b', 6), ('a', 5), ('c', 12)]) 
 OrderedDict([('b', '6'), ('a', '5'), ('c', '12')])
OrderedDict([('b', 6), ('a', 10), ('c', 12)]) 
 OrderedDict([('b', '6'), ('a', '10'), ('c', '12')])
b 6
c 12
5
6


In [4]:
import time

a = np.full(100000, float('nan'))
b = np.random.random(a.shape)

def set_inplace(b, a):
    a[:] = b + 2
    return a

def set_notinplace(b):
    return b + 2

set_inplace(b, a)

t1 = time.time()
for iter in range(1000):
    set_inplace(b, a)
print('Inplace: ', time.time() - t1)

t1 = time.time()
for iter in range(1000):
    a = set_notinplace(b)
print('Not in place: ', time.time() - t1)

# WTF?  Try %memit and id

Inplace:  0.0842735767364502
Not in place:  0.03564643859863281


In [5]:
import LinearResponseVariationalBayes as lrvb
import LinearResponseVariationalBayes.base_parameters as paragami

def check_equal(a, b, tol=1e-9):
    return np.linalg.norm(a - b) < tol

ImportError: No module named 'LinearResponseVariationalBayes.base_parameters'

In [ ]:
test_shape = (2, 3, 4)
a = np.random.random(test_shape)

array_pattern = paragami.ArrayPattern('a', test_shape, lb=-1, ub=2)

a_flat = array_pattern.flatten(a, free=False)
print(a_flat)
a_fold = array_pattern.fold(a_flat, free=False)
assert check_equal(a, a_fold)

a_flat = array_pattern.flatten(a, free=True)
print(a_flat)
a_fold = array_pattern.fold(a_flat, free=True)
assert check_equal(a, a_fold)


In [ ]:
dict_pattern = paragami.OrderedDictPattern('dict')

dict_pattern['a'] = paragami.ArrayPattern('a', (2, 3, 4), lb=-1, ub=2)
dict_pattern['b'] = paragami.ArrayPattern('b', (5, ), lb=-1, ub=10)

dict_val = dict_pattern.random()
print(dict_val['a'])
print(dict_val['b'])

for free in [True, False]:
    dict_flat = dict_pattern.flatten(dict_val, free=free)
    dict_fold = dict_pattern.fold(dict_flat, free=free)
    for key  in dict_fold:
        assert check_equal(dict_fold[key], dict_val[key])
